## Codice per estrarre embeddings esm da una cartella contenente file .fasta

In [1]:
from pathlib import Path
import torch
import esm
import time
import gc
import argparse
from esm import Alphabet, FastaBatchedDataset, ProteinBertModel, pretrained
import os

In [2]:
def extract_embeddings(model_name, fasta_file, output_dir, tokens_per_batch=4096, seq_length=1022,repr_layers=[33]):
    
    
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    # Extract the directory name from the fasta_file path
    fasta_file_dir = Path(fasta_file).parent.name
    # Construct output file name based on the fasta_file directory name
    filename = output_dir / f"{fasta_file_dir}_proteins_embeddings.pt"

    
    model, alphabet = pretrained.load_model_and_alphabet(model_name)
    model.eval()

    if torch.cuda.is_available():
        model = model.cuda()
        
    dataset = FastaBatchedDataset.from_file(fasta_file)
    batches = dataset.get_batch_indices(tokens_per_batch, extra_toks_per_seq=1)

    data_loader = torch.utils.data.DataLoader(
        dataset, 
        collate_fn=alphabet.get_batch_converter(seq_length), 
        batch_sampler=batches
    )

    output_dir.mkdir(parents=True, exist_ok=True)

    all_results = {}  # Initialize container for all results
    
    with torch.no_grad():
        for batch_idx, (labels, strs, toks) in enumerate(data_loader):

            print(f'Processing batch {batch_idx + 1} of {len(batches)}')

            if torch.cuda.is_available():
                toks = toks.to(device="cuda", non_blocking=True)

            out = model(toks, repr_layers=repr_layers, return_contacts=False)

            logits = out["logits"].to(device="cpu")
            representations = {layer: t.to(device="cpu") for layer, t in out["representations"].items()}
            
            for i, label in enumerate(labels):
                entry_id = label.split()[0]
                
                #filename = output_dir / f"{entry_id}.pt"
                truncate_len = min(seq_length, len(strs[i]))

                result = {"entry_id": entry_id}
                result["mean_representations"] = {
                        layer: t[i, 1 : truncate_len + 1].mean(0).clone()
                        for layer, t in representations.items()
                    }
                all_results[entry_id] = result  # Collect results in dictionary

                torch.save(all_results, filename)



In [3]:
model_name = 'esm2_t30_150M_UR50D'
#fasta_file = ("C:/Users/lorenzo/Desktop/phagescope/FASTA/protein_fasta/RefSeq/NC_000866.4/NC_000866.4_combined.fasta")
fasta_file = ("C:/Users/lorenzo/Desktop/phagescope/FASTA/protein_fasta/AAA/NC_000866.4/NC_000866.4_combined.fasta")
output_dir = ("C:/Users/lorenzo/Desktop")

extract_embeddings(model_name, fasta_file, output_dir, repr_layers=list(range(0,33)), tokens_per_batch=4096)

Processing batch 1 of 9
Processing batch 2 of 9
Processing batch 3 of 9
Processing batch 4 of 9
Processing batch 5 of 9


OutOfMemoryError: CUDA out of memory. Tried to allocate 206.00 MiB. GPU 

In [11]:
list(range(1,10))

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [4]:
e = torch.load("C:/Users/lorenzo/Desktop/embds/NC_000866.4_proteins_embeddings.pt")
#e = e['mean_representations'].numpy()
#type(e['mean_representations'][5])
#print(e[NP_049878.1])

In [6]:
e[30]

KeyError: 30

In [3]:
model_name = 'esm2_t30_150M_UR50D'
base_directory = "C:/Users/lorenzo/Desktop/phagescope/FASTA/protein_fasta/AAA"
times = []
for root, dirs, files in os.walk(base_directory):
    for file in files:
        file_path = os.path.join(root, file)
        #print(file_path)
        #dir_name = os.path.basename(root)
        #print(root)
        temp = time.time()
        extract_embeddings(model_name, file_path, root, repr_layers=[30])
        times.append(time.time() - temp)

Processing batch 1 of 16
Processing batch 2 of 16
Processing batch 3 of 16
Processing batch 4 of 16
Processing batch 5 of 16
Processing batch 6 of 16
Processing batch 7 of 16
Processing batch 8 of 16
Processing batch 9 of 16
Processing batch 10 of 16
Processing batch 11 of 16
Processing batch 12 of 16
Processing batch 13 of 16
Processing batch 14 of 16
Processing batch 15 of 16
Processing batch 16 of 16
Processing batch 1 of 2
Processing batch 2 of 2
Processing batch 1 of 5
Processing batch 2 of 5
Processing batch 3 of 5
Processing batch 4 of 5
Processing batch 5 of 5
Processing batch 1 of 5
Processing batch 2 of 5
Processing batch 3 of 5
Processing batch 4 of 5
Processing batch 5 of 5
Processing batch 1 of 5
Processing batch 2 of 5
Processing batch 3 of 5
Processing batch 4 of 5
Processing batch 5 of 5
Processing batch 1 of 7
Processing batch 2 of 7
Processing batch 3 of 7
Processing batch 4 of 7
Processing batch 5 of 7
Processing batch 6 of 7
Processing batch 7 of 7
Processing batch 

In [4]:
import statistics
print(statistics.mean(times))
print("estimated_time " + (str(statistics.mean(times) * 4636/60/60*13)) +" hours")

6.1709901094436646
estimated_time 103.3092310877641 hours


In [23]:
#len(e)
print(e["NP_049878.1"]["mean_representations"][30])

278

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#!pip install scikit-learn
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Convert list of tensors to a single tensor
data_tensor = torch.stack(sequence_representations)

# Convert PyTorch tensor to NumPy array
data = data_tensor.numpy()


# Step 1: Standardize the data
scaler = StandardScaler()
data_standardized = scaler.fit_transform(data)

# Step 2: Perform PCA
pca = PCA(n_components=2)  # Reducing to 2 components for 2D plotting
principal_components = pca.fit_transform(data_standardized)

# Step 3: Plot the results
plt.figure(figsize=(8, 6))
plt.scatter(principal_components[:, 0], principal_components[:, 1], c='blue', marker='o')
plt.title('PCA of Numerical Vectors')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE


# Convert list of tensors to a 2D numpy array
data = np.array(torch.stack(sequence_representations))

# Apply t-SNE
tsne = TSNE(n_components=2, random_state=42)
data_2d = tsne.fit_transform(data)

# Plot the t-SNE results
plt.figure(figsize=(8, 6))
plt.scatter(data_2d[:, 0], data_2d[:, 1], c='blue', edgecolor='k')
plt.title('t-SNE visualization of tensors')
plt.xlabel('t-SNE component 1')
plt.ylabel('t-SNE component 2')
plt.show()


In [13]:
############# COMBINE FASTA FILES #############

import os

def combine_fasta_files(input_directory, output_file):
    seen_labels = set()
    with open(output_file, 'w') as outfile:
        for root, dirs, files in os.walk(input_directory):
            for file in files:
                if file.endswith(".fasta") or file.endswith(".fa"):
                    file_path = os.path.join(root, file)
                    with open(file_path, 'r') as infile:
                        write_data = False
                        for line in infile:
                            if line.startswith(">"):
                                label = line.strip()
                                if label not in seen_labels:
                                    seen_labels.add(label)
                                    outfile.write(line)
                                    write_data = True
                                else:
                                    write_data = False
                            elif write_data:
                                outfile.write(line)
# Example usage
#input_directory = "C:/Users/lorenzo/Desktop/phagescope/FASTA/protein_fasta/RefSeq/NC_000866.4"
#output_file = 'C:/Users/lorenzo/Desktop/one_bacter_proteins.fasta'
#combine_fasta_files(input_directory, output_file)


In [38]:
import os
input_directory = "C:/Users/lorenzo/Desktop/phagescope/FASTA/protein_fasta/RefSeq"
output_file = 'C:/Users/lorenzo/Desktop/one_bacter_proteins.fasta'

In [39]:
def delete_fasta_files(input_directory, exclude_file):
    print ("exclude_file: " + exclude_file)
    for root, dirs, files in os.walk(input_directory):
        for file in files:
            if not ("combined" in file):
                os.remove(os.path.join(root, file))
                print("deleting : " + file)

def process_subfolders_recursively(base_directory):
    for root, dirs, files in os.walk(base_directory):
        # Only process subfolders, skip the root folder
        for subdir in dirs:
            subfolder_path = os.path.join(root, subdir)
            output_file = os.path.join(subfolder_path, f"{subdir}_combined.fasta")
            combine_fasta_files(subfolder_path, output_file)
            delete_fasta_files(subfolder_path, output_file)

In [8]:



############# COMBINE FASTA FILES #############
# Sostituisce tutti i singoli file .fasta con un unico file _combined.fasta

import os

input_directory = "C:/Users/lorenzo/Desktop/phagescope/FASTA/protein_fasta/RefSeq"

def combine_fasta_files(input_directory, output_file):
    seen_labels = set()
    with open(output_file, 'w') as outfile:
        for root, dirs, files in os.walk(input_directory):
            for file in files:
                if file.endswith(".fasta") or file.endswith(".fa"):
                    file_path = os.path.join(root, file)
                    with open(file_path, 'r') as infile:
                        write_data = False
                        for line in infile:
                            if line.startswith(">"):
                                label = line.strip()
                                if label not in seen_labels:
                                    seen_labels.add(label)
                                    outfile.write(line)
                                    write_data = True
                                else:
                                    write_data = False
                            elif write_data:
                                outfile.write(line)

def delete_fasta_files(input_directory, exclude_file):
    #print ("exclude_file: " + exclude_file)
    for root, dirs, files in os.walk(input_directory):
        for file in files:
            if not ("combined" in file):
                os.remove(os.path.join(root, file))
                #print("deleting : " + file)

def process_subfolders_recursively(base_directory):
    for root, dirs, files in os.walk(base_directory):
        # Only process subfolders, skip the root folder
        for subdir in dirs:
            subfolder_path = os.path.join(root, subdir)
            output_file = os.path.join(subfolder_path, f"{subdir}_combined.fasta")
            combine_fasta_files(subfolder_path, output_file)
            delete_fasta_files(subfolder_path, output_file)



In [12]:
############# COMBINE FASTA FILES FAST #############

import os
import concurrent.futures

#input_directory = "C:/Users/lorenzo/Desktop/phagescope/FASTA/protein_fasta/RefSeq"

def process_file(file_path, seen_labels):
    combined_data = []
    with open(file_path, 'r') as infile:
        write_data = False
        for line in infile:
            if line.startswith(">"):
                label = line.strip()
                if label not in seen_labels:
                    seen_labels.add(label)
                    combined_data.append(line)
                    write_data = True
                else:
                    write_data = False
            elif write_data:
                combined_data.append(line)
    return combined_data

def combine_fasta_files(input_directory, output_file):
    seen_labels = set()
    combined_data = []

    # Use ThreadPoolExecutor to process files in parallel
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for root, dirs, files in os.walk(input_directory):
            for file in files:
                if file.endswith(".fasta") or file.endswith(".fa"):
                    file_path = os.path.join(root, file)
                    futures.append(executor.submit(process_file, file_path, seen_labels))
        
        for future in concurrent.futures.as_completed(futures):
            combined_data.extend(future.result())
    
    with open(output_file, 'w') as outfile:
        outfile.writelines(combined_data)

def delete_fasta_files(input_directory, exclude_file):
    for root, dirs, files in os.walk(input_directory):
        for file in files:
            if not "combined" in file and (file.endswith(".fasta") or file.endswith(".fa")):
                file_path = os.path.join(root, file)
                os.remove(file_path)
                #print(f"Deleting: {file}")

def process_subfolders_recursively_fast(base_directory):
    for root, dirs, files in os.walk(base_directory):
        # Only process subfolders, skip the root folder
        for subdir in dirs:
            subfolder_path = os.path.join(root, subdir)
            output_file = os.path.join(subfolder_path, f"{subdir}_combined.fasta")
            combine_fasta_files(subfolder_path, output_file)
            delete_fasta_files(subfolder_path, output_file)

#process_subfolders_recursively(input_directory)


In [10]:

import time

input_directory = "C:/Users/lorenzo/Desktop/phagescope/FASTA/protein_fasta/AAA"
start = time.time()
process_subfolders_recursively(input_directory)
print("elapsed normal: " +  str(time.time() - start))



elapsed normal: 12.826902151107788


In [14]:
input_directory = "C:/Users/lorenzo/Desktop/phagescope/FASTA/protein_fasta/RefSeq"
start = time.time()
process_subfolders_recursively_fast(input_directory)
print("elapsed fast: " +  str(time.time() - start))

elapsed fast: 638.7071475982666
